In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

# Load the CSV file
data = pd.read_csv('data/merged_file.csv')

# We're only interested in Job_Title and Job_Details columns
data = data[['Job_Title', 'Job_Details']]

# Preprocess the job details and job titles (filling any missing values)
data['Job_Details'] = data['Job_Details'].fillna('')
data['Job_Title'] = data['Job_Title'].fillna('')

# TF-IDF Vectorization for job details and job titles
vectorizer_details = TfidfVectorizer(max_features=5000, stop_words='english')
X_details = vectorizer_details.fit_transform(data['Job_Details']).toarray()

vectorizer_titles = TfidfVectorizer(max_features=1000, stop_words='english')  # Vectorize job titles as well
X_titles = vectorizer_titles.fit_transform(data['Job_Title']).toarray()

# The target is the job details, which we are trying to enhance
y = X_details  # The output is the enhanced job details

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_titles, y, test_size=0.2, random_state=42)

# Define a basic Neural Network model in TensorFlow
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Input is job title vector
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(y_train.shape[1], activation='linear')  # Output is enhanced job details (resume content)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Save the model for later use
model.save('resume_enhanced_generator_model.h5')

# Function to enhance resume based on job title
def enhance_resume(job_title, model, vectorizer_titles, vectorizer_details):
    # Vectorize the job title input
    job_title_vectorized = vectorizer_titles.transform([job_title]).toarray()

    # Predict the enhanced job details (resume content)
    predicted_resume_content = model.predict(job_title_vectorized)

    # Inverse transform the predicted content back to text
    enhanced_resume_text = vectorizer_details.inverse_transform(predicted_resume_content)

    return ' '.join(enhanced_resume_text[0])

# Example usage: Provide a job title to enhance the resume
job_title_input = "Data Scientist"
enhanced_resume = enhance_resume(job_title_input, model, vectorizer_titles, vectorizer_details)

print(f"Enhanced Resume for Job Title '{job_title_input}':\n{enhanced_resume}")


FileNotFoundError: [Errno 2] No such file or directory: 'data/merged_file.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')